# Capstone Project - The Battle of the Neighborhoods (Week 4)

## Best neighborhood to operate an independant coffee shop in Toronto

### Introduction

This is a "Capstone project" for the final course in the IBM Data Science Professional Certificate, Weeks 4/5. This project aims to determine the best location to open an independent coffee shop using Python scripts, visualization libraries, pandas & numpy packages, FourSquare API, etc.

This project aims to enable aspiring enterpreneures in making an informed choice in terms of location identification by taking into account various factors like population demographics, existing competitors and so on.

### Description and Background of the Problem

For this project, Toronto has been chosen as it has a growing independent coffee shop scene. According to Wikipedia - "Toronto is the capital city of the Canadian province of Ontario. With a recorded population of 2,731,571 in 2016, it is the most populous city in Canada and the fourth most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,245,438 people (as of 2016) surrounding the western end of Lake Ontario, while the Greater Toronto Area (GTA) proper had a 2016 population of 6,417,516. Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world."

As it is known, choosing a neighborhood and location for a coffee shop is an important and diffucult step for the entrepreneurs. Most of time it can be as crucial factor as it determines the coffee shop's success and also affects the type of service, quality, menu, interior design.

### Description of Data

To identify the best location, the following data is required:

1. The list of Toronto neighborhoods represented by postal codes and their boroughs.(https://tinyurl.com/toronto-postal-code)

2. The most updated record of traffic signal vehicle and pedestrian volumes in Toronto.(https://tinyurl.com/vehicle-foot-traffic)

3. The popular or venues of a given neighborhood in Toronto.(https://developer.foursquare.com/)

The List of neighborhood data will be used to obtain the exact coordinates for each neighborhood based on the postal code, this allows exploration of the city map.
Then the coordinates and Foursquare credentials will be used to access the popular venues along with their details, especially for coffee shops. The pedestrian volumes can be used to analyze the foot traffic.